In [1]:
import os
import sys
import numpy as np
import pandas as pd
import itertools
from tqdm import tqdm
import time

# Set up path to project root
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(project_root)

# Custom modules
from src.problems.ising import relaxed_ising_energy, grad_relaxed_ising, ising_energy
from src.optimizers.sa import sa_continuous, sa_discrete
from src.optimizers.gd import gradient_descent
from src.optimizers.hybrid import sa_gd_hybrid
from src.utils.utils_experiments import bootstrap_experiment_ising, evaluate_continuous_results, evaluate_ising_results

# Output directory
results_dir = os.path.join(project_root, "results", "gridsearch")
os.makedirs(results_dir, exist_ok=True)

In [ ]:
# Hyperparameter grids
sa_grid = {
    'T_init': [10, 50, 100],
    'alpha': [0.9, 0.95, 0.99, 0.995, 0.999],
    'step_size': [0.005, 0.01, 0.05, 0.1, 0.3, 0.5]
}

sa_grid_cont = {
    'T0': [10, 50, 100],
    'alpha': [0.9, 0.95, 0.99, 0.995, 0.999],
    'step_size': [0.005, 0.01, 0.05, 0.1, 0.3, 0.5]
}

gd_grid = {
    'lr': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.3, 0.5]
}

lattice_shape = (10, 10)
num_runs = 25
dim_ising = 10
x_inits_disc = [np.random.choice([-1, 1], size=lattice_shape) for _ in range(num_runs)]
ascent_method = "unif"

In [3]:
def run_one_sa_discrete(T_init, alpha, num_runs=25):
    # Define known ground-truth state for evaluation
    best_state = np.ones(lattice_shape)

    # Set up initial spin configurations
    x_inits_disc = [
        np.random.choice([-1, 1], size=lattice_shape)
        for _ in range(num_runs)
    ]

    # Run bootstrap experiment
    result = bootstrap_experiment_ising(
        algorithm_function=sa_discrete,
        runs=num_runs,
        f=ising_energy,
        dim=dim_ising,
        x_inits=x_inits_disc,
        is_discrete=True,
        best_state=best_state,
        hamming_threshold=0,
        T_init=T_init,
        alpha=alpha,
        lattice_size=lattice_shape,
        max_iter=20000,
        tol=1e-6,
        f_star=-200.0
    )

    stats = result["stats"]
    
    return {
        "algorithm": "SA_discrete",
        "T_init": T_init,
        "alpha": alpha,
        "mean": stats.get("mean"),
        "best": stats.get("best"),
        "worst": stats.get("worst"),
        "std": stats.get("std"),
        "rmse" : stats.get("rmse"),
        "mse" : stats.get("rmse"),
        "mean_runtime": stats.get("mean_runtime"),
        "hamming_best_to_gt": stats.get("hamming_best_to_gt")
    }

def grid_search_sa_discrete(grid, num_runs=25):
    combos = list(itertools.product(grid["T_init"], grid["alpha"]))
    results = []

    for idx, (T_init, alpha) in enumerate(tqdm(combos), 1):
        print(f"[{idx}/{len(combos)}] T_init = {T_init}, alpha = {alpha}")
        res = run_one_sa_discrete(T_init, alpha, num_runs)
        results.append(res)

    return pd.DataFrame(results)

In [4]:
def run_one_sa_continuous(T0, alpha, step_size, num_runs=25):

    # define known ground-truth state for evaluation
    best_state = np.ones(lattice_shape)

    x_inits_cont = [
        np.random.uniform(low=-1.0, high=1.0, size=lattice_shape)
        for _ in range(num_runs)
    ]

    result = bootstrap_experiment_ising(
        algorithm_function=sa_continuous,
        runs=num_runs,
        f=relaxed_ising_energy,
        grad_f=None,
        dim=dim_ising,
        x_inits=x_inits_cont,
        is_discrete=False,
        T0=T0,
        alpha=alpha,
        step_size=step_size,
        max_iter=20000,
        tol=1e-6,
        f_star=-200.0,
        init_range = (-1,1)
    )

    stats = result["stats"]

    return {
        "algorithm": "SA_continuous",
        "T0": T0,
        "alpha": alpha,
        "step_size": step_size,
        "mean": stats.get("mean"),
        "best": stats.get("best"),
        "worst": stats.get("worst"),
        "std": stats.get("std"),
        "rmse": stats.get("rmse"),
        "mse": stats.get("mse"),
        "mean_runtime": stats.get("mean_runtime"),
        "mean_distance_to_x_star": stats.get("mean_distance_to_x_star")
    }

def grid_search_sa_continuous(grid, num_runs=25):
    combos = list(itertools.product(grid["T0"], grid["alpha"], grid["step_size"]))
    results = []

    for idx, (T0, alpha, step_size) in enumerate(tqdm(combos), 1):
        print(f"[{idx}/{len(combos)}] T0 = {T0}, alpha = {alpha}, step_size = {step_size}")
        res = run_one_sa_continuous(T0, alpha, step_size, num_runs)
        results.append(res)

    return pd.DataFrame(results)

In [5]:
def run_one_gd_ising_continuous(lr, num_runs=25):
    best_state = np.ones(lattice_shape)

    x_inits_cont = [
        np.random.uniform(low=-1.0, high=1.0, size=lattice_shape)
        for _ in range(num_runs)
    ]
    x_inits_flat = [x.flatten() for x in x_inits_cont]

    # Wrap f and grad_f to handle flattened input
    relaxed_f = lambda x: relaxed_ising_energy(x.reshape(lattice_shape))
    grad_wrapped = lambda x: grad_relaxed_ising(x.reshape(lattice_shape)).flatten()

    result = bootstrap_experiment_ising(
        algorithm_function=gradient_descent,
        runs=num_runs,
        f=relaxed_f,
        grad_f=grad_wrapped,
        dim=dim_ising,
        x_inits=x_inits_flat,
        is_discrete=False,
        lr=lr,
        max_iter=20000,
        tol=1e-6,
        f_star=-200.0,
        init_range=(-1, 1)
    )

    stats = result["stats"]
    return {
        "algorithm": "GD_continuous",
        "lr": lr,
        "mean": stats.get("mean"),
        "best": stats.get("best"),
        "worst": stats.get("worst"),
        "std": stats.get("std"),
        "rmse": stats.get("rmse"),
        "mse": stats.get("mse"),
        "mean_runtime": stats.get("mean_runtime"),
        "mean_distance_to_x_star": stats.get("mean_distance_to_x_star")
    }


def grid_search_gd_ising_continuous(grid, num_runs=25):
    results = []

    for idx, lr in enumerate(tqdm(grid["lr"]), 1):
        print(f"[{idx}/{len(grid['lr'])}] lr = {lr}")
        res = run_one_gd_ising_continuous(lr, num_runs)
        results.append(res)

    return pd.DataFrame(results)



# Running actual Gridsearch

In [6]:
# print("\nRunning SA-discrete on ising_energy")
# df_sa_discrete = grid_search_sa_discrete(sa_grid, num_runs=25)
# df_sa_discrete.to_csv(os.path.join(results_dir, "gridsearch_sa_discrete_ising_new.csv"), index=False)

In [7]:
# print("\nRunning SA-continuous on relaxed_ising")
# df_sa_continuous = grid_search_sa_continuous(sa_grid_cont, num_runs=25)
# df_sa_continuous.to_csv(os.path.join(results_dir, "gridsearch_sa_continuous_ising_new.csv"), index=False)

In [8]:
# print("\nRunning GD on relaxed_ising_energy")
# df_gd = grid_search_gd_ising_continuous(gd_grid, num_runs=1)
# df_gd.to_csv(os.path.join(results_dir, "gridsearch_gd_continuous_ising_new.csv"), index=False)

# Hybrid

In [10]:
# Define the grid
hybrid_grid = {
    'lr': [0.0001, 0.001, 0.01, 0.05, 0.1],
    'sigma': [0.1, 0.5, 1.0, 2.0],
    'T0': [0.01, 0.1, 1.0, 10.0]
}

In [11]:
def run_one_hybrid_ising_relaxed(lr, sigma, T0, num_runs=25):
    best_state = np.ones(lattice_shape)

    # generate flat initializations
    x_inits = [
        np.random.uniform(low=-1.0, high=1.0, size=lattice_shape).flatten()
        for _ in range(num_runs)
    ]

    # wrap the function and gradient
    relaxed_f = lambda x: relaxed_ising_energy(x.reshape(lattice_shape))
    grad_wrapped = lambda x: grad_relaxed_ising(x.reshape(lattice_shape)).flatten()

    result = bootstrap_experiment_ising(
        algorithm_function=sa_gd_hybrid,
        f=relaxed_f,
        grad_f=grad_wrapped,
        runs=num_runs,
        dim=dim_ising,
        x_inits=x_inits,
        is_discrete=False,
        name="ising_relaxed",
        lr=lr,
        sigma=sigma,
        T0=T0,
        max_iter=20000,
        tol=1e-6,
        f_star=-200.0,
        init_range=(-1, 1),
    )

    stats = result["stats"]
    return {
        "algorithm": "Hybrid_SA_GD",
        "lr": lr,
        "sigma": sigma,
        "T0": T0,
        "mean": stats.get("mean"),
        "best": stats.get("best"),
        "worst": stats.get("worst"),
        "std": stats.get("std"),
        "rmse": stats.get("rmse"),
        "mse": stats.get("mse"),
        "rmed": stats.get("rmed"),
        "mean_runtime": stats.get("mean_runtime"),
        "mean_distance_to_x_star": stats.get("mean_distance_to_x_star")
    }

def grid_search_hybrid_ising_relaxed(grid, num_runs=25):
    combos = list(itertools.product(grid["lr"], grid["sigma"], grid["T0"]))
    results = []

    for idx, (lr, sigma, T0) in enumerate(tqdm(combos), 1):
        print(f"[{idx}/{len(combos)}] lr = {lr}, sigma = {sigma}, T0 = {T0}")
        res = run_one_hybrid_ising_relaxed(lr, sigma, T0, num_runs)
        results.append(res)

    return pd.DataFrame(results)


In [13]:
# Run and save
df_hybrid_ising = grid_search_hybrid_ising_relaxed(hybrid_grid, num_runs=10)
df_hybrid_ising.to_csv(os.path.join(results_dir, "gridsearch_hybrid-unif_ising_relaxed_new.csv"), index=False)

  0%|          | 0/80 [00:00<?, ?it/s]

[1/80] lr = 0.0001, sigma = 0.1, T0 = 0.01
[Ising Relaxed] Run 1/10
[Ising Relaxed] Run 2/10
[Ising Relaxed] Run 3/10
[Ising Relaxed] Run 4/10
[Ising Relaxed] Run 5/10
[Ising Relaxed] Run 6/10
[Ising Relaxed] Run 7/10
[Ising Relaxed] Run 8/10
[Ising Relaxed] Run 9/10
[Ising Relaxed] Run 10/10


  1%|▏         | 1/80 [03:57<5:12:28, 237.32s/it]

[2/80] lr = 0.0001, sigma = 0.1, T0 = 0.1
[Ising Relaxed] Run 1/10
[Ising Relaxed] Run 2/10
[Ising Relaxed] Run 3/10
[Ising Relaxed] Run 4/10
[Ising Relaxed] Run 5/10
[Ising Relaxed] Run 6/10
[Ising Relaxed] Run 7/10
[Ising Relaxed] Run 8/10
[Ising Relaxed] Run 9/10
[Ising Relaxed] Run 10/10


  2%|▎         | 2/80 [07:12<4:35:58, 212.29s/it]

...

 98%|█████████▊| 78/80 [5:41:59<08:17, 248.84s/it]

[79/80] lr = 0.1, sigma = 2.0, T0 = 1.0
[Ising Relaxed] Run 1/10
[Ising Relaxed] Run 2/10
[Ising Relaxed] Run 3/10
[Ising Relaxed] Run 4/10
[Ising Relaxed] Run 5/10
[Ising Relaxed] Run 6/10
[Ising Relaxed] Run 7/10
[Ising Relaxed] Run 8/10
[Ising Relaxed] Run 9/10
[Ising Relaxed] Run 10/10


 99%|█████████▉| 79/80 [5:45:42<04:00, 240.92s/it]

[80/80] lr = 0.1, sigma = 2.0, T0 = 10.0
[Ising Relaxed] Run 1/10
[Ising Relaxed] Run 2/10
[Ising Relaxed] Run 3/10
[Ising Relaxed] Run 4/10
[Ising Relaxed] Run 5/10
[Ising Relaxed] Run 6/10
[Ising Relaxed] Run 7/10
[Ising Relaxed] Run 8/10
[Ising Relaxed] Run 9/10
[Ising Relaxed] Run 10/10


100%|██████████| 80/80 [5:49:44<00:00, 262.30s/it]
